In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import tensorflow as tf

2022-03-12 16:13:52.356445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from scripts.python.nn.generate_embeddings import load_context_encoder

2022-03-12 16:14:06,532 - faiss.loader - INFO: Loading faiss with AVX2 support.
2022-03-12 16:14:06,533 - faiss.loader - INFO: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-03-12 16:14:06,533 - faiss.loader - INFO: Loading faiss.
2022-03-12 16:14:06,603 - faiss.loader - INFO: Successfully loaded faiss.


In [5]:
from transformers import TFRobertaModel
from transformers import PhobertTokenizer
from libs.nn.modeling import DualEncoder

In [6]:
query_encoder = TFRobertaModel.from_pretrained("pretrained/vinai/phobert-base")
context_encoder = TFRobertaModel.from_pretrained("pretrained/vinai/phobert-base")
dual_encoder = DualEncoder(query_encoder=query_encoder, context_encoder=context_encoder)
tokenizer = PhobertTokenizer.from_pretrained("pretrained/vinai/phobert-base")

Some layers from the model checkpoint at pretrained/vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at pretrained/vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at pretrained/vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This I

In [72]:
query = "phạm_vi điều_chỉnh của luật đấu_thầu"
context = {
    "title": "trách_nhiệm của nhà_thầu kiểm_toán :",
    "text": "nhà_thầu kiểm_toán , kiểm_toán_viên khi thực_hiện kiểm_toán quyết_toán dự_án hoàn_thành phải chấp_hành nguyên_tắc hoạt_động kiểm_toán độc_lập , có quyền_hạn , nghĩa_vụ và chịu trách_nhiệm theo quy_định của pháp_luật về kiểm_toán ;"
}

In [7]:
import json
with open("data/v2/train_data.json", "r") as reader:
    data = json.load(reader)

In [64]:
context = data[10]["positive_contexts"][0]
query = data[10]["questions"][0]
hardneg_contexts = data[3200]["positive_contexts"][:20]

In [21]:
from libs.data_helpers.build_tfrecord import tokenize_batched_contexts

In [73]:
context_inputs = tokenize_batched_contexts(tokenizer, [context], 256)
context_inputs = {
    "input_ids": tf.convert_to_tensor(context_inputs["input_ids"]),
    "attention_mask": tf.convert_to_tensor(context_inputs["attention_mask"])
}
# hardneg_contexts_inputs = tokenize_batched_contexts(tokenizer, hardneg_contexts, 256)
# hardneg_contexts_inputs = {
#     "input_ids": tf.convert_to_tensor(hardneg_contexts_inputs["input_ids"]),
#     "attention_mask": tf.convert_to_tensor(hardneg_contexts_inputs["attention_mask"])
# }
query_inputs = tokenizer(query, return_tensors="tf")

In [53]:
ckpt = tf.train.Checkpoint(model=dual_encoder)
ckpt.restore("checkpoints/luffy/ckpt-50")

In [74]:
query_embedding = query_encoder(**query_inputs, return_dict=True, training=False).last_hidden_state[:, 0, :]
context_embedding = context_encoder(**context_inputs, return_dict=True, training=False).last_hidden_state[:, 0, :]
# hardneg_context_embedding = context_encoder(**hardneg_contexts_inputs, return_dict=True, training=False).last_hidden_state[:, 0, :]

In [67]:
# combine_context_embedding = tf.concat([context_embedding, hardneg_context_embedding], axis=0)
# sim_matrix = tf.matmul(query_embedding, combine_context_embedding, transpose_b=True)
# print(tf.squeeze(sim_matrix).numpy())
# sim_probs = tf.nn.softmax(sim_matrix, axis=-1)
# print(tf.squeeze(sim_probs).numpy())

[34.85981   -7.384517  -5.1384287 -5.1301684 -4.1229057]
[1.0000000e+00 4.5032043e-19 4.2558480e-18 4.2911421e-18 1.1749563e-17]


In [75]:
tf.reduce_sum(query_embedding * context_embedding)

<tf.Tensor: shape=(), dtype=float32, numpy=32.02346>